In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.86498,"""a""",0.375252,0.881995,0.420171,0.655727,0.173962,-0.309318
0.841471,1,"""a""",1,0.50387,"""a""",0.463363,0.40801,0.082464,0.50011,0.666425,0.068278
0.909297,2,"""a""",0,0.477584,"""a""",0.830754,0.432502,0.467936,0.052073,0.918524,-0.447444
0.14112,3,"""a""",1,0.6773,"""a""",0.494657,0.382809,0.988702,0.970549,0.782791,-1.29399
-0.756802,4,"""a""",0,0.132401,"""a""",0.851374,0.198486,0.783481,0.237095,0.512069,-0.987913


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.655727,-0.173962
-0.841471,-0.50011,-0.666425
-0.909297,-0.052073,-0.918524
-0.14112,-0.970549,-0.782791
0.756802,0.418632,-0.338106


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[-0.504084, -0.336834]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.216516,0.001691,128.048013,0.0,0.213201,0.21983
"""exp(x2)""",0.175173,0.000677,258.937137,0.0,0.173847,0.176499
"""sin(x3)""",-1.74255,0.001332,-1308.318803,0.0,-1.745161,-1.739939
"""__bias__""",-0.108934,0.001506,-72.347151,0.0,-0.111886,-0.105983


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[-0.504084, -0.336834]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.500149, -0.342264]"
"""a""","[-0.500149, -0.342264]"
"""a""","[-0.500149, -0.342264]"
"""a""","[-0.500149, -0.342264]"
"""a""","[-0.500149, -0.342264]"
…,…
"""b""","[-0.508067, -0.331294]"
"""b""","[-0.508067, -0.331294]"
"""b""","[-0.508067, -0.331294]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.375252,0.881995,-0.309318,-0.486245,0.176927
0.463363,0.40801,0.068278,-0.371006,0.439283
0.830754,0.432502,-0.447444,-0.564452,0.117008
0.494657,0.382809,-1.29399,-0.378292,-0.915698
0.851374,0.198486,-0.987913,-0.496021,-0.491892


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.508067, -0.331294]"
"""a""","[-0.500149, -0.342264]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.321225, -0.172559]"
"""b""","[-0.338743, -0.153431]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.query_lstsq(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.526524
"""b""",-0.5288


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.query_rolling_lstsq(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.309318,0.375252,0.881995,null,null
0.068278,0.463363,0.40801,null,null
-0.447444,0.830754,0.432502,null,null
-1.29399,0.494657,0.382809,null,null
-0.987913,0.851374,0.198486,"[-1.027403, 0.107542]",-0.853358
…,…,…,…,…
-1.062121,0.567943,0.940267,"[0.000874, -1.220791]",-1.147372
-1.225309,0.729136,0.208833,"[-1.048079, -0.461308]",-0.860528
-1.05447,0.201154,0.844424,"[-1.234935, -0.568308]",-0.728305


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[28.935948, 28.819625, 28.595828]"


In [18]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.905711,"[0.947839, 0.318751]"
28.809196,"[-0.318751, 0.947839]"


In [19]:
# PC1
df.select(
    pds.query_principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
0.042494
0.051967
-0.292343
0.534958
-0.24653


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.332623,1.009241,0.516993,0.503505,0.51016,0.516366,0.505279
"""a""",0.335301,1.006624,0.492925,0.514356,0.503412,0.494169,0.49816


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""hello"""
"""going"""
"""church"""
"""to"""
"""world"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""hello"""
"""go"""
"""world"""
""""""
"""church"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [30]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [31]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-1.402932
-1.259118
1.917719


In [32]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,0.319249,null
null,-0.556602,null
-1.402932,1.333906,-1.471444
-1.259118,-1.171916,0.680943
1.917719,0.556111,0.603619


In [33]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""TA1""",null
null,"""gmH""",null
-1.402932,"""9cy""","""dW"""
-1.259118,"""kavq""","""a"""
1.917719,"""7X""","""uUWp"""


In [34]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-1.402932,"""l3LAK"""
-1.259118,"""ZuJYd"""
1.917719,"""gsQlg"""


In [35]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.597323,null,-0.597315
null,-0.603972,null,-0.60384
-1.402932,1.237547,1.620943,1.237837
-1.259118,0.314343,0.008727,0.314366
1.917719,0.297839,3.227741,0.29761


In [36]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.733562,1.097824,3
null,0.137709,0.559485,0
-1.402932,0.822002,0.107044,3
-1.259118,0.678968,-0.861539,1
1.917719,0.349169,0.917781,0


In [37]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
1.415335,0.157179,3.687327,0.158237


In [38]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.874512,0.532159,0,3
1,0.182455,0.849439,3,4
2,0.129595,0.998405,3,7
0,0.890222,0.848518,3,3
1,0.184214,0.88866,2,1


In [39]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.382819,0.701862}","{30.300258,0.736013}","{1.302779,0.266475}"


In [40]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.query_chi2("category_1", "category_2").alias("chi2-test"),
        pds.query_f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬─────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test              ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                 ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]           ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪═════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-0.59735,0.550314} ┆ {50.692756,0.053027} ┆ {0.185401,0.946065} │
│ 1         ┆ {0.750403,0.453065} ┆ {34.32586,0.548339}  ┆ {1.868009,0.113496} │
│ 2         ┆ {0.518838,0.603908} ┆ {35.215448,0.505716} ┆ {1.393921,0.233671} │
└───────────┴─────────────────────┴──────────────────────┴─────────────────────┘


In [41]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
511,0.1022
545,0.109
545,0.109
575,0.115
571,0.1142
562,0.1124
574,0.1148
544,0.1088
573,0.1146


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [42]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [43]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.520135,0.023887,0.424417,0.616866,7.166066,9
1,0.362396,0.6973,0.919444,0.291983,6.803053,12
2,0.208545,0.823846,0.981867,0.663537,7.166631,14
3,0.909944,0.970713,0.45529,0.290241,6.290227,5
4,0.924506,0.957152,0.86393,0.261618,8.396401,14


In [44]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.520135,0.023887,0.424417,0.616866,7.166066,309
1,0.362396,0.6973,0.919444,0.291983,6.803053,52
2,0.208545,0.823846,0.981867,0.663537,7.166631,293
3,0.909944,0.970713,0.45529,0.290241,6.290227,47
4,0.924506,0.957152,0.86393,0.261618,8.396401,25


In [45]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.520135,0.023887,0.424417,0.616866,7.166066,"[0, 321, … 1795]"
1,0.362396,0.6973,0.919444,0.291983,6.803053,"[1, 1156, … 1536]"
2,0.208545,0.823846,0.981867,0.663537,7.166631,"[2, 1752, … 1374]"
3,0.909944,0.970713,0.45529,0.290241,6.290227,"[3, 875, … 1469]"
4,0.924506,0.957152,0.86393,0.261618,8.396401,"[4, 1537, … 1375]"


In [46]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 321, … 678]   ┆ 8                  │
│ 1   ┆ [1, 1156, … 1028] ┆ 7                  │
│ 2   ┆ [2, 1752, … 1454] ┆ 6                  │
│ 3   ┆ [3, 875]          ┆ 1                  │
│ 4   ┆ [4, 1537, … 1932] ┆ 10                 │
└─────┴───────────────────┴────────────────────┘


In [47]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.520135,0.023887,0.424417,0.616866,7.166066,"[0, 321, … 1795]","[0.0, 0.058375, … 0.067274]"
1,0.362396,0.6973,0.919444,0.291983,6.803053,"[1, 1156, … 1536]","[0.0, 0.04118, … 0.058766]"
2,0.208545,0.823846,0.981867,0.663537,7.166631,"[2, 1752, … 1374]","[0.0, 0.028817, … 0.054837]"
3,0.909944,0.970713,0.45529,0.290241,6.290227,"[3, 875, … 1469]","[0.0, 0.090139, … 0.109005]"
4,0.924506,0.957152,0.86393,0.261618,8.396401,"[4, 1537, … 1375]","[0.0, 0.039451, … 0.051298]"


In [48]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
5,0.811302,0.599191,0.710414,0.537152,3.457735
6,0.75459,0.461001,0.381041,0.722947,7.324939
12,0.558716,0.414653,0.465225,0.079993,8.504178
13,0.60055,0.284698,0.299793,0.085367,2.727863
16,0.788044,0.478008,0.674275,0.450806,0.39686


In [49]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
226,0.421302,0.471938,0.284193,0.049393,0.128264
229,0.575597,0.541939,0.631847,0.052552,2.287534
249,0.485648,0.424313,0.560687,0.032285,0.088083
306,0.502801,0.551935,0.603083,0.44726,4.7951
323,0.540203,0.488882,0.053305,0.969637,7.19677


In [50]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
323,0.540203,0.488882,0.053305,0.969637,7.19677
366,0.50812,0.546394,0.037332,0.713331,8.360705
543,0.517303,0.458524,0.872152,0.727479,6.32662
718,0.559907,0.496147,0.850939,0.255172,8.788911
763,0.511012,0.529572,0.27042,0.26353,8.113711


In [51]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 946]",2
1,"[1, 1156, 1241]",3
2,"[2, 1540, 185]",3
3,"[3, 628, … 4]",6
4,"[4, 1537, … 3]",6


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [52]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [53]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [54]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [55]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [56]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"


# Using PDS Expressions On Series / NumPy arrays

In [57]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.506222,2,4
0.0,0.783347,0,7
1.0,0.447991,0,6
0.0,0.97702,0,0
0.0,0.813408,1,1


In [58]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "query_jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [59]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.500998,0.50192,0.501459,0.500656,0.501317


In [60]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "query_psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.216072,0.2,0.232,0.004749
0.411602,0.2,0.194,0.000183
0.590313,0.2,0.174,0.003621
0.790307,0.2,0.188,0.000743
inf,0.2,0.212,0.000699


In [61]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
12.975626


In [62]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.124653


# Examples using Ball Tree Features

Not currently available.

In [63]:
# size = 1_000
# df = pl.DataFrame({
#     "f": np.sin(list(range(size)))
#     , "time_idx": range(size)
#     , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
#     , "actual": np.round(np.random.random(size=size)).astype(np.int32)
#     , "predicted": np.random.random(size=size)
#     , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
# }).with_columns(
#     pds.random(0., 1.).alias("x1")
#     , pds.random(0., 1.).alias("x2")
#     , pds.random(0., 1.).alias("x3")
#     , pds.random(0., 1.).alias("a")
#     , pds.random(0., 1.).alias("b")
# ).with_columns(
#     y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
# )
# df = df.with_columns(pl.int_range(0, size).cast(dtype=pl.UInt32).alias("index"))
# df.head()

In [64]:
# # Pointwise Nearest Neighbors
# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         parallel=True).alias("ball_tree_knn_ptwise")
# )

In [65]:
# # Pointwise Nearest Neighbors with distances
# # we get a struct with an ids column and a dist column

# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         return_dist=True,
#         parallel=True).alias("ball_tree_knn_ptwise")
# ).unnest("ball_tree_knn_ptwise")

In [66]:
# # Frequency Count
# df.with_columns(
#     pds.query_bt_knn_radius_freq_cnt(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =5, 
#         distance_metric="haversine", 
#         parallel=True
#         ).alias("ball_tree_knn_radius_freq_cnt")
# )

In [67]:
# # Average distance of k nearest neighbors
# df.with_columns(
#     pds.query_bt_knn_avg(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         pl.col("x1"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =1,
#         distance_metric="euclidean", 
#         parallel=True).alias("btree_knn_avg")
# )

In [68]:
# # Count neighbors within radius
# df.with_columns(
#     pds.query_bt_nb_cnt(
#         pl.col("x2"),
#         pl.col("x3"),
#         index=pl.col("index"),
#         r=18.0, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_radius")
# )

In [69]:
# # returns true if the row is within k nearest neighbors of the given point
# df.with_columns(
#     pds.bt_within_dist_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=[123,78.99],
#         r=999999, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_dist_from")
# )

In [70]:
# # check if a point is included in the k nearest neighbors of the given point
# # Note: This is an exact match so your point could be quite close but now show up
# # We can pass in an EPSILON to account for this. THis is defaulted to the EPSILON of the data type in Rust

# # pick a random point
# point = df.select(pl.col("x2"), pl.col("x3"))
# rr = list(point.row(2))
# df.with_columns(
#     pds.is_bt_knn_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=rr, 
#         k=56, 
#         distance_metric="haversine",
#         parallel=True, epsilon=0.5).alias("btree_nn_within")
# )